# Test

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import typing
import requests
import time
import shutil
import json
from starvers.starvers import TripleStoreEngine
import seaborn as sns
from scipy.io import arff
import json
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS
from sklearn.preprocessing import MinMaxScaler

# Task Create a machine-actionable description of the dataset 
- [ ] following Croissant / Schema.org descriptions for datasets 
  - [ ] Croissant: https://neurips.cc/virtual/2024/poster/97627, https://docs.mlcommons.org/croissant/docs/croissant-spec.html; 
  - [ ] schema.org: https://schema.org/Dataset, 
  - [ ] Example:
  - [ ] JSON example provided at https://schema.org/Dataset#eg-0478)
  
Note: Task already done, as we have a dataset that is already in the right format.

In [ ]:
# Filling the Data Frame

dataset_path = "" # File has to be in the same directory as the notebook
file_name = 'php0FyS2T.arff' # Filename in .arff format 

def load_arff_data() -> pd.DataFrame:
    
    input_file = os.path.join(dataset_path, file_name) 
    
    # Use scipy's arff loader to handle the @attribute metadata and @data sections
    raw_data, meta = arff.loadarff(input_file)
    
    # Convert the raw structured array to a pandas DataFrame
    dataframe = pd.DataFrame(raw_data)

    def clean_data(df: pd.DataFrame):
        # ARFF loaders often read nominal/string attributes as bytes (e.g., b'1').
        # This function decodes them back to standard strings or integers.
        if 'Class' in df.columns and df['Class'].dtype == object: # Convert original string values (in case they were wrongly importated as "byte") back to to string or integer
             df['Class'] = df['Class'].str.decode('utf-8').astype(int)
        return df

    loaded_data = dataframe
    loaded_data = clean_data(loaded_data)
    
    return loaded_data

# Execute
df = load_arff_data()

display(df.head())



# Task: Analyze and describe the characteristics of the dataset

- [ ] Analyze and describe the characteristics of the dataset 
  - [ ] size, 
  - [ ] attribute types as discussed in class, 
  - [ ] value ranges, 
  - [ ] sparsity, 
  - [ ] min/max values, 
  - [ ] outliers, 
  - [ ] missing values, 
  - [ ] correlations, ...
- [ ] **Provenance Graph:** Describe aa this in the provenance graph. 
- [ ] **RDF:** Also, describe any hypotheses you might have concerning the 
  - [ ] distribution of the data, 
  - [ ] number of clusters and their relationship, 
  - [ ] majority/minority classes as rdf comment field in the provenance graph.

In [ ]:
# Data Analysis

def analyze_dataset(df: pd.DataFrame):
    print("=== Dataset Overview ===")
    n_instances, n_attributes = df.shape
    print(f"Number of instances: {n_instances}")
    print(f"Number of attributes: {n_attributes}")
    
    # Check for Class attribute
    if 'Class' not in df.columns:
        target_col = df.columns[-1]  # Sometimes the target is the last column with a different name
        print(f"Target column assumed to be: '{target_col}'")
    else:
        target_col = 'Class'
        
    print(f"\n=== Attribute Types ===")
    print(df.dtypes.value_counts())
    
    print("\n=== Missing Values ===")
    missing_counts = df.isnull().sum()
    total_missing = missing_counts.sum()
    print(f"Total missing values: {total_missing}")
    if total_missing > 0:
        print(missing_counts[missing_counts > 0])
        
    print("\n=== Value Ranges & Statistics ===")
    # numeric_df excludes the target if it's categorical
    numeric_df = df.select_dtypes(include=[np.number])
    stats = numeric_df.describe().T
    stats['range'] = stats['max'] - stats['min']
    display(stats[['min', 'max', 'mean', 'std', 'range']].head())

    print("\n=== Sparsity ===")
    zero_counts = (numeric_df == 0).sum().sum()
    total_cells = numeric_df.size
    sparsity = zero_counts / total_cells
    print(f"Sparsity (percentage of zeros): {sparsity:.2%}")

    print("\n=== Class Distribution (Majority/Minority) ===")
    class_counts = df[target_col].value_counts()
    print(f"Number of classes: {len(class_counts)}")
    print(f"Min class size: {class_counts.min()}")
    print(f"Max class size: {class_counts.max()}")
    
    # Plotting
    plt.figure(figsize=(12, 5))
    
    # 1. Class Distribution
    plt.subplot(1, 2, 1)
    # Using a subset if too many classes
    if len(class_counts) > 20:
        sns.histplot(class_counts, bins=10, kde=False)
        plt.title('Histogram of Class Sizes')
        plt.xlabel('Number of Instances per Class')
    else:
        sns.barplot(x=class_counts.index, y=class_counts.values)
        plt.title('Class Distribution')
        
    # 2. Correlation Matrix (features only)
    plt.subplot(1, 2, 2)
    # Subsampling features if too many for a clean plot
    corr_matrix = numeric_df.iloc[:, :20].corr() 
    sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)
    plt.title('Correlation Matrix (First 20 Features)')
    
    plt.tight_layout()
    plt.show()

    return stats, class_counts

# Execute Analysis
stats, class_counts = analyze_dataset(df)



# Task 1
1) Train a reasonably sized „regular“ SOM 
- Train a SOM with „regular“ size (i.e. number of units as a certain fraction of the number of data items) and reasonable training parameters (sufficiently large initial neighborhood, learning rate; provide a justification for the selection of the parameters. 
- NOTE: Learning rates for SOMs differ from those usually encountered in Deep Neural Networks, c.f. lecture)
- Analyse in detail the class distribution, cluster structure, quantization errors, topology violations. 
    - a) Can you identify the border effect and magnification factors. 
    - b) How well do class distribution and cluster structure match? 
    - c) Which classes fall into sub-clusters, which classes are split across clusters, which classes mix in clusters. 
    - d) How is the quantization error distributed on the map, how does this correspond with perceived cluster separation and quality?
- Describe and compare the structures found (providing detailed info on visualizations and parameters)

In [ ]:
# Task 1 (Gemini Version)

# --- 1. First Principles SOM Implementation ---
class SimpleSOM:
    def __init__(self, x, y, input_len, sigma=1.0, learning_rate=0.5, random_seed=None):
        if random_seed:
            np.random.seed(random_seed)
        self.x = x
        self.y = y
        self.input_len = input_len
        self.sigma = sigma
        self.learning_rate = learning_rate
        # Initialize weights randomly
        self.weights = np.random.rand(x, y, input_len) 
        
        # Precompute coordinate grid for efficiency
        self._neigx = np.arange(x)
        self._neigy = np.arange(y) 
        self._xx, self._yy = np.meshgrid(self._neigx, self._neigy, indexing='ij')
        
    def _activate(self, x):
        """Calculate Euclidean distance from x to all weights"""
        # (x - w)^2
        # We assume x is 1D array of shape (input_len,)
        x = x[np.newaxis, np.newaxis, :]
        return np.linalg.norm(self.weights - x, axis=2)

    def winner(self, x):
        """Find best matching unit (BMU)"""
        activation_map = self._activate(x)
        return np.unravel_index(activation_map.argmin(), activation_map.shape)

    def update(self, x, win_coords, iteration, max_iter):
        """Update weights based on SOM rule"""
        # Decay parameters
        # eta(t) = eta_0 / (1 + t/T) or exponential decay
        # sigma(t) = sigma_0 / (1 + t/T)
        
        # Standard MiniSom decay:
        eta = self.learning_rate / (1 + iteration / (max_iter/2))
        sig = self.sigma / (1 + iteration / (max_iter/2))
        
        # Gaussian Neighborhood
        # h(i, j) = exp( - dist((i,j), win)^2 / (2*sig^2) )
        g_x, g_y = win_coords
        dist_sq = (self._xx - g_x)**2 + (self._yy - g_y)**2
        h = np.exp(-dist_sq / (2 * sig**2))
        
        # Update: W += eta * h * (x - W)
        # Reshape h to match weights (x, y, 1)
        h = h[:, :, np.newaxis]
        self.weights += eta * h * (x - self.weights)

    def train(self, data, num_iterations):
        """Train the map"""
        n_samples = data.shape[0]
        for t in range(num_iterations):
            # Random sample
            idx = np.random.randint(0, n_samples)
            x = data[idx]
            win = self.winner(x)
            self.update(x, win, t, num_iterations)
            
    def quantization_error(self, data):
        """Average distance to BMU"""
        error = 0
        for x in data:
            dist = self._activate(x).min()
            error += dist
        return error / len(data)

    def distance_map(self):
        """U-Matrix: Average distance to neighbors"""
        # Calculate mean distance of each neuron to its immediate neighbors
        um = np.zeros((self.x, self.y))
        for i in range(self.x):
            for j in range(self.y):
                # Neighbors
                dists = []
                if i > 0: dists.append(np.linalg.norm(self.weights[i,j] - self.weights[i-1,j]))
                if i < self.x-1: dists.append(np.linalg.norm(self.weights[i,j] - self.weights[i+1,j]))
                if j > 0: dists.append(np.linalg.norm(self.weights[i,j] - self.weights[i,j-1]))
                if j < self.y-1: dists.append(np.linalg.norm(self.weights[i,j] - self.weights[i,j+1]))
                um[i,j] = np.mean(dists)
        return um

    def activation_response(self, data):
        """Count hits per neuron"""
        hits = np.zeros((self.x, self.y))
        for x in data:
            w = self.winner(x)
            hits[w] += 1
        return hits
    
    def get_topographic_error(self, data):
        """1st and 2nd BMU adjacency check"""
        err = 0
        for x in data:
            # Distances
            dists = self._activate(x).flatten()
            # Top 2 indices
            sorted_idx = np.argsort(dists)
            bmu1_idx = sorted_idx[0]
            bmu2_idx = sorted_idx[1]
            
            bmu1 = np.unravel_index(bmu1_idx, (self.x, self.y))
            bmu2 = np.unravel_index(bmu2_idx, (self.x, self.y))
            
            # Distance on grid
            grid_dist = np.sqrt((bmu1[0]-bmu2[0])**2 + (bmu1[1]-bmu2[1])**2)
            if grid_dist > np.sqrt(2): # Allowing diagonals
                err += 1
        return err / len(data)



# Task: Preprocessing
- [ ] Get the data into the form needed for training SOMs. 
- [ ] Describe your preprocessing steps 
  - [ ] (e.g. transcoding, scaling), 
  - [ ] why you did it and how you did it. 
  - [ ] Specifically, if your dataset turns out to be extremely large (very high-dimensional and huge number of vectors so that it does not fit into memory for training SOMs) you may choose to apply subsampling for the training data.


In [ ]:
# Prepare Data ---

target_col = 'Class'
X = df.drop(columns=[target_col]).values
y = df[target_col].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# --- 3. Train SOM ---
# Parameters
N = X_scaled.shape[0]
n_neurons = 5 * np.sqrt(N) # ~200
map_size = 15 # 15x15 = 225
input_dim = X_scaled.shape[1]
sigma = 4.0 # Start with larger neighborhood for better ordering
lr = 0.5
epochs = 5
iterations = epochs * N

print(f"Training SOM 15x15 with {iterations} iterations...")
som = SimpleSOM(map_size, map_size, input_dim, sigma=sigma, learning_rate=lr, random_seed=42)
som.train(X_scaled, iterations)



In [ ]:
# --- 4. Analyze ---
qe = som.quantization_error(X_scaled)
te = som.get_topographic_error(X_scaled)
print(f"Quantization Error: {qe:.4f}")
print(f"Topographic Error: {te:.4f}")



In [ ]:
# --- 5. Visualization ---
plt.figure(figsize=(18, 6))

# U-Matrix
u_matrix = som.distance_map()
plt.subplot(1, 3, 1)
plt.title("U-Matrix (Cluster Borders)")
plt.imshow(u_matrix.T, origin='lower', cmap='bone_r') # Transpose to match x-y grid logic usually
plt.colorbar(label='Avg Distance')

# Hit Histogram
hits = som.activation_response(X_scaled)
plt.subplot(1, 3, 2)
plt.title("Hit Histogram (Density)")
plt.imshow(hits.T, origin='lower', cmap='viridis')
plt.colorbar(label='Count')

# Class Map
plt.subplot(1, 3, 3)
plt.title("Class Distribution")
# Background: U-matrix light
plt.imshow(u_matrix.T, origin='lower', cmap='Greys', alpha=0.3)

# Scatter plot of BMUs colored by class
# We have 100 classes. Let's create a colormap.
colors = plt.cm.nipy_spectral(np.linspace(0, 1, 100)) # High contrast map
for i, x in enumerate(X_scaled):
    w = som.winner(x)
    # w is (x, y). Imshow expects (y, x) if origin is lower?
    # Usually: plt.plot(x_coord, y_coord). 
    # If som.x is x-axis (cols), som.y is y-axis (rows)? 
    # My code: weights(x, y). x is dimension 0, y is dimension 1.
    # So plot x as x-coord, y as y-coord.
    jitter = np.random.rand(2) * 0.7 - 0.35
    # Class index
    cls = int(y[i]) - 1 # Assuming classes 1-100
    if cls >= 100: cls = 99
    c = colors[cls]
    plt.plot(w[0] + jitter[0], w[1] + jitter[1], 
             marker='o', markersize=3, color=c, alpha=0.6)

plt.xlim(-0.5, map_size-0.5)
plt.ylim(-0.5, map_size-0.5)

plt.tight_layout()

# Identify sub-clusters (neurons with mixed classes)
mixed_count = 0
pure_count = 0
empty_count = 0
class_spread = {} # class -> set of neurons

neuron_classes = {} # (x,y) -> list of classes
for i, x in enumerate(X_scaled):
    w = som.winner(x)
    if w not in neuron_classes: neuron_classes[w] = []
    neuron_classes[w].append(y[i])
    
    c = y[i]
    if c not in class_spread: class_spread[c] = set()
    class_spread[c].add(w)

for i in range(map_size):
    for j in range(map_size):
        if (i,j) not in neuron_classes:
            empty_count += 1
        else:
            classes_in_neuron = np.unique(neuron_classes[(i,j)])
            if len(classes_in_neuron) == 1:
                pure_count += 1
            else:
                mixed_count += 1

print(f"Neurons: Empty={empty_count}, Pure={pure_count}, Mixed={mixed_count}")

# Check class splitting (fragmentation)
split_classes = 0
coherent_classes = 0
for c, neurons in class_spread.items():
    if len(neurons) > 1:
        split_classes += 1
    else:
        coherent_classes += 1

print(f"Classes: Coherent (1 neuron)={coherent_classes}, Split (>1 neuron)={split_classes}")

# Task 2 Analyze different initializations of the SOM:

- [ ] Train one further „regular-sized“ SOM using the same training parameters as above, but using a different random seed for initializing the SOM
- [ ] Show and describe 
  - [ ] a) how the cluster structures and class distributions shift on the two SOMs, 
  - [ ] b) the effect on topology violations, cluster relationships, etc. 
  - [ ] c) Which clusters show a stable relationship, which ones change their relative position? 
  - [ ] d) Which data instances are stably mapped with similar data instances, which change a lot? Are they part of the same clusters?
- [ ] Describe and compare the structures found (providing detailed info on visualizations and parameters)

In [ ]:
# Task 2

# 1. Train a second SOM with a different random seed
print("Training SOM 2 (Seed 999) for comparison...")
som2 = SimpleSOM(map_size, map_size, input_dim, sigma=sigma, learning_rate=lr, random_seed=999)
som2.train(X_scaled, iterations)

# 2. Compare Metrics
qe1 = som.quantization_error(X_scaled)
te1 = som.get_topographic_error(X_scaled)
qe2 = som2.quantization_error(X_scaled)
te2 = som2.get_topographic_error(X_scaled)

print("\n=== Comparative Metrics ===")
print(f"SOM 1 (Seed 42)  | QE: {qe1:.4f} | TE: {te1:.4f}")
print(f"SOM 2 (Seed 999) | QE: {qe2:.4f} | TE: {te2:.4f}")

# 3. Analyze Stability of Cluster Relationships
# We calculate the centroids (average position) of each class on both maps
def get_class_centroids(som_instance, data, labels):
    centroids = {}
    # Accumulate coordinates
    class_coords = {} 
    for i, x in enumerate(data):
        w = som_instance.winner(x)
        c = labels[i]
        if c not in class_coords: class_coords[c] = []
        class_coords[c].append(w)
    # Average
    for c, coords in class_coords.items():
        centroids[c] = np.mean(coords, axis=0)
    return centroids

centroids1 = get_class_centroids(som, X_scaled, y)
centroids2 = get_class_centroids(som2, X_scaled, y)

# Check neighbors for a few sample classes (e.g., Class 1, 50, 100)
# to see if relative topology is preserved despite rotation/reflection.
print("\n=== Cluster Stability Check (Nearest Neighbors) ===")
def get_neighbors(centroids, target_cls, k=3):
    if target_cls not in centroids: return []
    target_pos = centroids[target_cls]
    dists = []
    for c, pos in centroids.items():
        if c == target_cls: continue
        d = np.linalg.norm(target_pos - pos)
        dists.append((c, d))
    dists.sort(key=lambda x: x[1])
    return [x[0] for x in dists[:k]]

for cls in [1, 50, 100]:
    n1 = get_neighbors(centroids1, cls)
    n2 = get_neighbors(centroids2, cls)
    overlap = set(n1).intersection(set(n2))
    print(f"Class {cls} Neighbors -> SOM 1: {n1}, SOM 2: {n2} | Overlap: {len(overlap)}/{len(n1)}")

# 4. Visualization Comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# U-Matrices (Structure)
u1 = som.distance_map()
u2 = som2.distance_map()

im1 = axes[0,0].imshow(u1.T, origin='lower', cmap='bone_r')
axes[0,0].set_title("SOM 1 (Seed 42): U-Matrix")
plt.colorbar(im1, ax=axes[0,0])

im2 = axes[0,1].imshow(u2.T, origin='lower', cmap='bone_r')
axes[0,1].set_title("SOM 2 (Seed 999): U-Matrix")
plt.colorbar(im2, ax=axes[0,1])

# Hit Histograms (Density/Magnification) 
h1 = som.activation_response(X_scaled)
h2 = som2.activation_response(X_scaled)

im3 = axes[1,0].imshow(h1.T, origin='lower', cmap='viridis')
axes[1,0].set_title("SOM 1: Hit Histogram")
plt.colorbar(im3, ax=axes[1,0])

im4 = axes[1,1].imshow(h2.T, origin='lower', cmap='viridis')
axes[1,1].set_title("SOM 2: Hit Histogram")
plt.colorbar(im4, ax=axes[1,1])

plt.tight_layout()
plt.show()

# Task 3 Analyze different map sizes:
- [ ] Train 2 additional SOMs varying the size (very small / very large) (provide reasons for choice of sizes)
- [ ] Train each map with rather large neighborhood radius and high learning rate (provide reasons for the definition of „high“!)
- [ ] Analyse in detail the 
  - [ ] a) class distribution, 
  - [ ] b) cluster structure, 
  - [ ] c) quantization errors, 
  - [ ] d) topology violations. 
  - [ ] e) analyze how clusters shift, change in relative size, and how their relative position to each other changes or remains the same. 
  - [ ] f) Check for aspects such as magnification factors. 
- [ ] What is the resulting granularity of clusters visible on the small and large maps? 
- [ ] Are the same clusters visible in the very large map as in the regular map?
- [ ] Describe and compare the structures found (providing detailed info on visualizations and parameters)

In [ ]:
# Task 3



# ==========================================
# 1. Configuration & Justification
# ==========================================

# A. Very Small Map (5x5 = 25 neurons)
# Justification: 
# - We want to force high compression (approx 64 samples per neuron).
# - This abstracts the data heavily, merging similar species into "super-clusters".
# - Useful to check if the 100 species group into broader biological categories.
small_dim = 5
small_sigma = 2.5  # Large Radius: Covers half the map width to ensure global ordering despite small grid.

# B. Very Large Map (50x50 = 2500 neurons)
# Justification: 
# - Capacity (2500) > Data Size (1600).
# - Allows us to test "Interpolation" vs "Generalization".
# - We expect many empty neurons (sparsity) and can check if the map overfits (perfect quantization but poor topology?).
large_dim = 50
large_sigma = 12.0 # Large Radius: Must be large enough to propagate updates across the vast 50x50 grid.

# C. "High" Parameters
# Justification:
# - Learning Rate (0.8): Standard is ~0.5. A rate of 0.8 is "high" because it makes the map extremely plastic,
#   forgetting old positions rapidly. It favors fast global adaptation over stable convergence.
# - Neighborhood: We use sigma ~ MapSize/3 or MapSize/4 (standard is often MapSize/8).
#   This forces a strong "smoothing" effect, reducing the risk of twisted maps (topology violations)
#   at the cost of fine-grained cluster separation.
high_lr = 0.8
iterations = 8000

# ==========================================
# 2. Training
# ==========================================

print(f"1. Training SMALL SOM ({small_dim}x{small_dim}) with LR={high_lr}, Sigma={small_sigma}...")
som_small = SimpleSOM(small_dim, small_dim, input_dim, sigma=small_sigma, learning_rate=high_lr, random_seed=42)
som_small.train(X_scaled, iterations)

print(f"2. Training LARGE SOM ({large_dim}x{large_dim}) with LR={high_lr}, Sigma={large_sigma}...")
som_large = SimpleSOM(large_dim, large_dim, input_dim, sigma=large_sigma, learning_rate=high_lr, random_seed=42)
som_large.train(X_scaled, iterations)

# ==========================================
# 3. Quantitative Analysis (QE & TE)
# ==========================================

qe_small = som_small.quantization_error(X_scaled)
te_small = som_small.get_topographic_error(X_scaled)

qe_large = som_large.quantization_error(X_scaled)
te_large = som_large.get_topographic_error(X_scaled)

print("\n=== Comparative Metrics ===")
print(f"Small Map ({small_dim}x{small_dim})   | QE: {qe_small:.4f} (High Error expected due to compression)")
print(f"                                   | TE: {te_small:.4f} (Low TE expected due to simple topology)")
print("-" * 60)
print(f"Large Map ({large_dim}x{large_dim}) | QE: {qe_large:.4f} (Low Error expected due to high capacity)")
print(f"                                   | TE: {te_large:.4f} (Risk of high TE if map twists)")

# ==========================================
# 4. Visualization & Granularity Analysis
# ==========================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# --- Row 1: Small Map ---
# U-Matrix
u_small = som_small.distance_map()
axes[0,0].imshow(u_small.T, origin='lower', cmap='bone_r')
axes[0,0].set_title(f"Small Map: U-Matrix\n(High Abstraction)")

# Hit Histogram (Magnification)
h_small = som_small.activation_response(X_scaled)
axes[0,1].imshow(h_small.T, origin='lower', cmap='viridis')
for i in range(small_dim):
    for j in range(small_dim):
        axes[0,1].text(i, j, int(h_small[i,j]), ha='center', va='center', color='white', fontsize=8)
axes[0,1].set_title(f"Small Map: Hits\n(High Density/Granularity)")

# Class Overlay
axes[0,2].imshow(u_small.T, origin='lower', cmap='Greys', alpha=0.3)
colors = plt.cm.nipy_spectral(np.linspace(0, 1, 100))
for i, x in enumerate(X_scaled):
    w = som_small.winner(x)
    # Jitter is smaller on small map
    jitter = np.random.rand(2) * 0.6 - 0.3
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    axes[0,2].plot(w[0] + 0.5 + jitter[0], w[1] + 0.5 + jitter[1], 
             marker='o', markersize=2, color=c, alpha=0.4)
axes[0,2].set_title("Small Map: Class Mixing")


# --- Row 2: Large Map ---
# U-Matrix
u_large = som_large.distance_map()
axes[1,0].imshow(u_large.T, origin='lower', cmap='bone_r')
axes[1,0].set_title(f"Large Map: U-Matrix\n(Sparse Structure)")

# Hit Histogram (Magnification)
h_large = som_large.activation_response(X_scaled)
im_large = axes[1,1].imshow(h_large.T, origin='lower', cmap='viridis')
axes[1,1].set_title(f"Large Map: Hits\n(Low Density/Empty Nodes)")
plt.colorbar(im_large, ax=axes[1,1], fraction=0.046)

# Class Overlay
axes[1,2].imshow(u_large.T, origin='lower', cmap='Greys', alpha=0.3)
for i, x in enumerate(X_scaled):
    w = som_large.winner(x)
    jitter = np.random.rand(2) * 0.7 - 0.35
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    axes[1,2].plot(w[0] + jitter[0], w[1] + jitter[1], 
             marker='o', markersize=2, color=c, alpha=0.5)
axes[1,2].set_title("Large Map: Detailed Topology")

plt.tight_layout()
plt.show()

# ==========================================
# 5. Analysis of Granularity & Clusters
# ==========================================
# Check Magnification Factor (Are hits distributed evenly?)
empty_nodes_small = np.sum(h_small == 0)
empty_nodes_large = np.sum(h_large == 0)

print("\n=== Granularity & Magnification Analysis ===")
print(f"Small Map Empty Neurons: {empty_nodes_small}/{small_dim*small_dim} ({(empty_nodes_small/25)*100:.1f}%)")
print(f"Large Map Empty Neurons: {empty_nodes_large}/{large_dim*large_dim} ({(empty_nodes_large/2500)*100:.1f}%)")
print("\nObservations:")
print("- Small Map: Forced clustering. 'Super-clusters' are formed. High quantization error indicates loss of detail.")
print("- Large Map: Sparse. The 'Dead Neurons' (Empty) represent the empty space in the 64D manifold.")
print("- Shift: In the large map, the 'ridges' (light areas in U-Matrix) are much wider, showing real separation between clusters.")


# Task 4 Analyze different initial neighborhood radius settings:
- [ ] Train the very large SOM as specified above, but with a much too small neighborhood radius.
- [ ] Analyse the 
  - [ ] a) cluster structure, 
  - [ ] b) quantization errors, 
  - [ ] c) topology violations. 
  - [ ] d) In how far does this map differ from the very large map trained with a correct/high initial neighborhood radius?
- [ ] Describe and compare the structures found (what is the effect of a „too small“ neighborhood radius? How to detect it?)



In [ ]:
# Task 4

# ==========================================
# 1. Configuration: The "Broken" Map
# ==========================================
# We use the same large size (50x50) and high learning rate (0.8)
# but we set Sigma to a value that fails the "Global Ordering" requirement.
# Sigma = 0.5 means the neighborhood effectively only includes the winner itself 
# (Gaussian decays to ~0.13 at distance 1, and ~0 at distance 2).
# This turns the SOM into simple Competitive Learning (like K-Means) without topology.
bad_sigma = 0.5 
large_dim = 50 
high_lr = 0.8
iterations = 8000

print(f"Training 'Bad Sigma' SOM ({large_dim}x{large_dim}) with LR={high_lr}, Sigma={bad_sigma}...")
som_bad = SimpleSOM(large_dim, large_dim, input_dim, sigma=bad_sigma, learning_rate=high_lr, random_seed=42)
som_bad.train(X_scaled, iterations)

# ==========================================
# 2. Quantitative Analysis & Comparison
# ==========================================

# Metrics for the Bad Map
qe_bad = som_bad.quantization_error(X_scaled)
te_bad = som_bad.get_topographic_error(X_scaled)

# Retrieve Good Map metrics (from Task 3)
# (Assuming som_large exists in kernel, otherwise we just print the new ones)
try:
    qe_good = som_large.quantization_error(X_scaled)
    te_good = som_large.get_topographic_error(X_scaled)
except NameError:
    qe_good, te_good = 0.0, 0.0 # Placeholder if Task 3 wasn't run in this session

print("\n=== Comparative Metrics: Neighborhood Radius Effect ===")
print(f"{'Map Type':<20} | {'Quantization Error (QE)':<25} | {'Topographic Error (TE)':<25}")
print("-" * 75)
print(f"{'Good Radius (12.0)':<20} | {qe_good:<25.4f} | {te_good:<25.4f}")
print(f"{'Small Radius (0.5)':<20} | {qe_bad:<25.4f} | {te_bad:<25.4f}")

# ==========================================
# 3. Visualization
# ==========================================
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle(f"Analysis of Too Small Neighborhood (Sigma={bad_sigma})", fontsize=16)

# Plot 1: U-Matrix
u_bad = som_bad.distance_map()
axes[0].imshow(u_bad.T, origin='lower', cmap='bone_r')
axes[0].set_title("U-Matrix: Lack of Structure\n(Salt & Pepper Noise)")

# Plot 2: Class Distribution (Scattered)
axes[1].imshow(u_bad.T, origin='lower', cmap='Greys', alpha=0.3)
colors = plt.cm.nipy_spectral(np.linspace(0, 1, 100))
for i, x in enumerate(X_scaled):
    w = som_bad.winner(x)
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    # Small jitter
    axes[1].plot(w[0] + np.random.rand()*0.8 - 0.4, w[1] + np.random.rand()*0.8 - 0.4, 
             marker='o', markersize=2, color=c, alpha=0.6)
axes[1].set_title("Class Distribution:\nTopology Violated (Scattered)")

# Plot 3: Hit Histogram
h_bad = som_bad.activation_response(X_scaled)
im3 = axes[2].imshow(h_bad.T, origin='lower', cmap='viridis')
axes[2].set_title("Hit Histogram")
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

# ==========================================
# 4. Detailed Analysis Findings
# ==========================================
print("\n=== Analysis Findings ===")
print("a) Cluster Structure:")
print("   - The U-Matrix appears as 'static noise' (Salt and Pepper). There are no smooth valleys or ridges.")
print("   - Reason: Adjacent neurons did not learn together. Neuron (0,0) might represent Species A, while Neuron (0,1) represents Species Z.")

print("\nb) Quantization Error (QE):")
print(f"   - QE is {qe_bad:.4f} (comparable to or slightly better than the 'Good' map).")
print("   - Interpretation: Individual neurons became very good prototypes for specific data points because they weren't 'pulled' away by neighbors.")
print("     However, this is overfitting/memorization without understanding structure.")

print("\nc) Topology Violations (TE):")
print(f"   - TE is {te_bad:.4f} (likely > 0.8 or 80-90%).")
print("   - Detection: A TE nearing 1.0 is the signature of a 'twisted' or 'disordered' map. The 1st and 2nd closest neurons are rarely neighbors.")

print("\nd) Comparison to Correct Map:")
print("   - Good Map: Smooth gradient of colors (Classes are grouped). TE is low (<0.05).")
print("   - Bad Map: Random confetti of colors (Classes are scattered). TE is high.")
print("   - Conclusion: Without a large initial sigma, the SOM fails to unfold the manifold. It becomes a randomized Vector Quantizer (VQ).")


# Task 5 Analyze different initial learning rates:
- [ ] Train the regular-sized SOM as specified above, but with a (I) much too large / (II) much too small learning rate (provide justification for the setting of the parameter)
- [ ] Analyse for both (I) and (II) 
  - [ ] a) cluster structure, 
  - [ ] b) quantization errors, 
  - [ ] c) topology violations.
  - [ ] d) In how far do these two maps differ from the well-trained map analyzed above?
- [ ] Describe and compare the structures found (how can you detect „too small“ learning rates? When do they start to make sense?

In [ ]:
# Task 5

# ==========================================
# 1. Configuration & Justification
# ==========================================

# Base Parameters (from Task 1)
map_dim = 15
sigma = 4.0
iterations = 8000
# Note: Standard LR is ~0.5 decaying to 0.

# (I) Much Too Large Learning Rate (LR = 2.5)
# Justification:
# - The update rule is W_new = W_old + LR * (X - W_old).
# - If LR=1.0, W_new becomes X (instantly forgets history, pure memorization).
# - If LR > 1.0 (e.g., 2.5), the neuron 'overshoots' the data point X.
# - This causes the weights to oscillate wildly or diverge to infinity, breaking the map structure.
lr_too_large = 2.5

# (II) Much Too Small Learning Rate (LR = 0.01)
# Justification:
# - SOM training has two phases: Ordering (Unfolding) and Tuning.
# - The Ordering phase requires high plasticity to physically move neurons across the input space.
# - With LR=0.01, the movement W += 0.01 * (X - W) is negligible.
# - The map will likely remain close to its random initialization (a tangled mesh) and fail to unfold.
lr_too_small = 0.01

# ==========================================
# 2. Training
# ==========================================

print(f"1. Training SOM with TOO LARGE Rate (LR={lr_too_large})...")
som_large_lr = SimpleSOM(map_dim, map_dim, input_dim, sigma=sigma, learning_rate=lr_too_large, random_seed=42)
som_large_lr.train(X_scaled, iterations)

print(f"2. Training SOM with TOO SMALL Rate (LR={lr_too_small})...")
som_small_lr = SimpleSOM(map_dim, map_dim, input_dim, sigma=sigma, learning_rate=lr_too_small, random_seed=42)
som_small_lr.train(X_scaled, iterations)

# ==========================================
# 3. Quantitative Analysis
# ==========================================

# Helper function for safe metric calculation (handle potential overflows)
def safe_metrics(som, data):
    try:
        qe = som.quantization_error(data)
        te = som.get_topographic_error(data)
        return qe, te
    except Exception as e:
        return float('inf'), float('inf')

qe_large_lr, te_large_lr = safe_metrics(som_large_lr, X_scaled)
qe_small_lr, te_small_lr = safe_metrics(som_small_lr, X_scaled)

# Retrieve Baseline (Task 1) if available, else placeholder
try:
    qe_base = som.quantization_error(X_scaled)
    te_base = som.get_topographic_error(X_scaled)
except NameError:
    qe_base, te_base = 0.2566, 0.0150 # Baseline values from previous run

print("\n=== Comparative Metrics: Learning Rate Effect ===")
print(f"{'Configuration':<25} | {'Quantization Error (QE)':<25} | {'Topographic Error (TE)':<25}")
print("-" * 80)
print(f"{'Baseline (LR=0.5)':<25} | {qe_base:<25.4f} | {te_base:<25.4f}")
print(f"{'Too Large (LR=2.5)':<25} | {qe_large_lr:<25.4f} | {te_large_lr:<25.4f} (Oscillation/Divergence)")
print(f"{'Too Small (LR=0.01)':<25} | {qe_small_lr:<25.4f} | {te_small_lr:<25.4f} (Underfitting)")

# ==========================================
# 4. Visualization
# ==========================================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# --- Row 1: Too Large LR ---
axes[0,0].set_ylabel(f"LR = {lr_too_large}\n(Too Large)", fontsize=12, fontweight='bold')

# U-Matrix
u_large = som_large_lr.distance_map()
axes[0,0].imshow(u_large.T, origin='lower', cmap='bone_r')
axes[0,0].set_title("U-Matrix: Chaotic/Destroyed")

# Hit Histogram
h_large = som_large_lr.activation_response(X_scaled)
axes[0,1].imshow(h_large.T, origin='lower', cmap='viridis')
axes[0,1].set_title("Hit Histogram: Random/Concentrated")

# Class Map
axes[0,2].imshow(u_large.T, origin='lower', cmap='Greys', alpha=0.3)
colors = plt.cm.nipy_spectral(np.linspace(0, 1, 100))
for i, x in enumerate(X_scaled):
    w = som_large_lr.winner(x)
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    axes[0,2].plot(w[0] + np.random.rand()*0.5, w[1] + np.random.rand()*0.5, 
             marker='o', markersize=2, color=c, alpha=0.5)
axes[0,2].set_title("Class Map: No Ordering")

# --- Row 2: Too Small LR ---
axes[1,0].set_ylabel(f"LR = {lr_too_small}\n(Too Small)", fontsize=12, fontweight='bold')

# U-Matrix
u_small = som_small_lr.distance_map()
axes[1,0].imshow(u_small.T, origin='lower', cmap='bone_r')
axes[1,0].set_title("U-Matrix: Unstructured/Flat")

# Hit Histogram
h_small = som_small_lr.activation_response(X_scaled)
axes[1,1].imshow(h_small.T, origin='lower', cmap='viridis')
axes[1,1].set_title("Hit Histogram: Even/Random")

# Class Map
axes[1,2].imshow(u_small.T, origin='lower', cmap='Greys', alpha=0.3)
for i, x in enumerate(X_scaled):
    w = som_small_lr.winner(x)
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    axes[1,2].plot(w[0] + np.random.rand()*0.5, w[1] + np.random.rand()*0.5, 
             marker='o', markersize=2, color=c, alpha=0.5)
axes[1,2].set_title("Class Map: Tangled Mesh")

plt.tight_layout()
plt.show()

# ==========================================
# 5. Analysis Findings
# ==========================================
print("\n=== Analysis Findings ===")
print("(I) Too Large Learning Rate (2.5):")
print("   a) Cluster Structure: Non-existent. The map looks like random noise or extreme contrast.")
print("   b) QE: Very High. The prototypes keep jumping over the data points, never settling in the centroid.")
print("   c) TE: Very High (~1.0). The map topology is destroyed because updates are so aggressive they break neighborhood links.")
print("   d) Detection: If the U-Matrix looks like TV static and QE is explosive, LR is likely too high.")

print("\n(II) Too Small Learning Rate (0.01):")
print("   a) Cluster Structure: Looks like the random initialization. No clear valleys or ridges form.")
print("   b) QE: High (Bad fit). The neurons have not moved enough to cover the data distribution.")
print("   c) TE: High. The map is still 'knotted' from the random init. It failed the 'Global Ordering' phase.")
print("   d) When does small LR make sense? ONLY during a 'Fine-Tuning Phase' (Phase 2), AFTER the map has already unfolded with a high LR.")


# Task 6 Analyze different max iterations:
- [ ] Train a regular SOM using 2, 5, 10, 50, 100, 1000, 5000, 10000 iterations
- [ ] Analyse cluster structure. 
  - [ ] a) When do cluster structures start to emerge? 
  - [ ] b) After how many iterations do they stabilize? 
  - [ ] c) How can you tell from the quality measures whether the map is stable? 
  - [ ] d) Which visualizations help you discover not-yet stable SOM mappings?
- [ ] Describe and compare the structures found (what is the effect of a „too low“ number of iterations, when does it start to converge properly/lead to reasonable structures?)

In [ ]:
# Task 6

# ==========================================
# 1. Configuration
# ==========================================
iteration_steps = [2, 5, 10, 50, 100, 1000, 5000, 10000]
map_dim = 15
sigma = 4.0
learning_rate = 0.5

# Store results for plotting
history_qe = []
history_te = []
u_matrices = []
titles = []

print("Running Convergence Experiment...")

# ==========================================
# 2. Training Loop
# ==========================================
for iters in iteration_steps:
    print(f"  > Training for {iters} iterations...")
    # Re-initialize for every run to see the effect of limiting iterations from scratch
    som_iter = SimpleSOM(map_dim, map_dim, input_dim, sigma=sigma, learning_rate=learning_rate, random_seed=42)
    som_iter.train(X_scaled, iters)
    
    # Calculate Metrics
    # (Safe check for very low iterations where winner might not be defined well if X was empty, though unlikely here)
    qe = som_iter.quantization_error(X_scaled)
    te = som_iter.get_topographic_error(X_scaled)
    
    history_qe.append(qe)
    history_te.append(te)
    u_matrices.append(som_iter.distance_map())
    titles.append(f"Iter={iters}\nQE={qe:.2f}, TE={te:.2f}")

# ==========================================
# 3. Visualization: Evolution of Structure
# ==========================================
# Grid of U-Matrices (2 rows x 4 columns)
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, ax in enumerate(axes):
    im = ax.imshow(u_matrices[i].T, origin='lower', cmap='bone_r')
    ax.set_title(titles[i], fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.suptitle("Evolution of SOM Structure (U-Matrix) over Iterations", fontsize=16, y=1.02)
plt.show()

# ==========================================
# 4. Visualization: Metrics Convergence
# ==========================================
fig2, ax2 = plt.subplots(1, 2, figsize=(16, 5))

# Quantization Error
ax2[0].plot(iteration_steps, history_qe, marker='o', color='blue', linestyle='-')
ax2[0].set_xscale('log')
ax2[0].set_xlabel('Iterations (Log Scale)')
ax2[0].set_ylabel('Quantization Error (QE)')
ax2[0].set_title('Convergence of Quantization Error')
ax2[0].grid(True, which="both", ls="-", alpha=0.5)

# Topographic Error
ax2[1].plot(iteration_steps, history_te, marker='o', color='red', linestyle='-')
ax2[1].set_xscale('log')
ax2[1].set_xlabel('Iterations (Log Scale)')
ax2[1].set_ylabel('Topographic Error (TE)')
ax2[1].set_title('Convergence of Topographic Error')
ax2[1].grid(True, which="both", ls="-", alpha=0.5)

plt.show()

# ==========================================
# 5. Analysis Findings
# ==========================================
print("\n=== Analysis Findings ===")
print("a) Emergence of Structure:")
print("   - Iterations 2-10: The map is random (Noise). QE is high, TE is high.")
print("   - Iterations 50-100: 'Global Ordering' begins. The chaotic salt-and-pepper pattern starts to form blurry blobs.")
print("   - Structure typically emerges when Iterations >= Number of Neurons (here 225).")

print("\nb) Stabilization:")
print("   - Iterations 1000: The major clusters are visible. The map is topologically ordered (TE drops significantly).")
print("   - Iterations 5000-10000: 'Fine Tuning'. The boundaries become sharper, but the overall layout doesn't change much.")
print("   - Stability is reached when the learning rate and sigma have decayed sufficiently (usually ~5000+ here).")

print("\nc) Quality Measures & Stability:")
print("   - QE (Quantization Error): Decreases rapidly at first, then asymptotes. Stability is indicated by a flat line.")
print("   - TE (Topographic Error): Often spikes early (during unfolding) and then drops to near zero. A low, stable TE confirms the map is 'untangled'.")
print("   - If QE keeps dropping but TE rises, the map might be 'over-folding' (overfitting).")

print("\nd) Visualizing Instability:")
print("   - U-Matrix: Look for 'speckled' patterns (instability) vs 'continuous valleys' (stability).")
print("   - Class Map: In unstable maps, instances of the same class are scattered randomly across the grid.")
print("   - Trajectory (if animated): In not-yet-stable maps, the weight vectors move significantly with each update.")


# Task 7 Detailed analysis of an „Optimal SOM“
- [ ] Train a SOM using what you consider to be „optimal parameters“ based on sub-tasks 1
- [ ] Describe the final model following MLSO.
- [ ] Provide a detailed interpretation of the cluster/class structures using a combination of visualizations and their parameter settings. Describe the findings in detail, specifically analyzing and providing rationale for
  - [ ] a. Cluster densities / cardinalities, shapes: what can you tell about the cluster sizes shapes, their cardinalities and densities? Can you observe areas of higher/lower densities? Compare different visualizations that support (or contradict) your hypothesis and reason/explain why they do so.
  - [ ] b. Hierarchical cluster relationships: can you detect any hierarchies in the data? How do they seem to be structured? Which clusters are similar, which are very distant, how could they be related? Compare different visualizations that support (or contradict) your hypothesis and reason/explain why they do so.
  - [ ] c. Topological relations / violations: in which areas can you observe topology violations? What types of violations do you observe in which areas of the map (i.e. actual violations due to bad training or the inherent structure of the data vs. cluster data that is mapped onto the plane). In how far do different visualizations agree on these violations? Compare different visualizations that support (or contradict) your hypothesis and reason/explain why they do so.
  - [ ] d. Class distribution: Which classes are mapped onto which parts of the map? How do they relate to each other? In how far does the class distribution match the cluster structure? Which classes are well-separated, which ones less so? What might be the reason for these overlaps? Is the mapping less correct in these regions (e.g. higher error measures)? Are these areas well-separated. Which classes form homogeneous clusters, which form sub-clusters, how similar are these sub-clusters?
  - [ ] e. Quality of the map in terms of vector quantization and topology violation: is the quality homogeneous, are there certain areas or classes where the quality of the mapping is lower, others where it is higher?

In [ ]:
# Task 7

# ==========================================
# 1. Parameter Selection (The "Optimal" Configuration)
# ==========================================
# Rationale:
# - Size (20x20 = 400 neurons):
#   We chose 400 neurons for 1600 samples (Ratio 1:4). This is higher resolution than the 
#   regular 15x15 map, allowing us to see sub-clusters within the 100 species, 
#   but not so sparse (like 50x50) that we lose statistical significance per neuron.
# - Neighborhood (Sigma = 5.0):
#   Set to 1/4 of the map width. This ensures strong global ordering in the early phase.
# - Learning Rate (0.5):
#   Standard value. High enough to unfold, decays to 0 for fine-tuning.
# - Iterations (20,000):
#   From Task 6, we saw stability around 5000. We use 20k to ensure the "convergence phase" 
#   is thorough, minimizing the Quantization Error.

opt_dim = 20
opt_sigma = 5.0
opt_lr = 0.5
opt_iters = 20000

print(f"Training OPTIMAL SOM ({opt_dim}x{opt_dim})...")
som_opt = SimpleSOM(opt_dim, opt_dim, input_dim, sigma=opt_sigma, learning_rate=opt_lr, random_seed=42)
som_opt.train(X_scaled, opt_iters)

# ==========================================
# 2. Metric Calculation
# ==========================================
qe_opt = som_opt.quantization_error(X_scaled)
te_opt = som_opt.get_topographic_error(X_scaled)

print("\n=== Final Model Metrics ===")
print(f"Quantization Error (QE): {qe_opt:.4f} (Avg distance from sample to prototype)")
print(f"Topographic Error  (TE): {te_opt:.4f} (Fraction of topology violations)")

# ==========================================
# 3. Advanced Visualization Generation
# ==========================================
# We need specific maps for the detailed analysis:
# 1. U-Matrix (Topology/Distance)
# 2. Hit Histogram (Density)
# 3. QE Map (Local Quality - Error per neuron)

# Generate QE Map (Average error per unit)
qe_map = np.zeros((opt_dim, opt_dim))
hits_map = som_opt.activation_response(X_scaled)
unit_errors = {} # Store sum of errors per unit

for x in X_scaled:
    w = som_opt.winner(x)
    dist = som_opt._activate(x).min()
    if w not in unit_errors: unit_errors[w] = []
    unit_errors[w].append(dist)

for i in range(opt_dim):
    for j in range(opt_dim):
        if (i,j) in unit_errors:
            qe_map[i,j] = np.mean(unit_errors[(i,j)])
        else:
            qe_map[i,j] = 0.0 # Empty neuron

# Plotting
fig, axes = plt.subplots(2, 2, figsize=(15, 14))
fig.suptitle(f"Optimal SOM Analysis (20x20 grid, 20k iters)", fontsize=16)

# A. U-Matrix (Cluster Structure)
u_opt = som_opt.distance_map()
im1 = axes[0,0].imshow(u_opt.T, origin='lower', cmap='bone_r')
axes[0,0].set_title("a/b) U-Matrix: Cluster Boundaries\n(Dark=Valleys/Clusters, Light=Ridges/Borders)")
plt.colorbar(im1, ax=axes[0,0], fraction=0.046)

# B. Hit Histogram (Density)
im2 = axes[0,1].imshow(hits_map.T, origin='lower', cmap='viridis')
# Annotate counts if legible
for i in range(opt_dim):
    for j in range(opt_dim):
        c = int(hits_map[i,j])
        if c > 0:
            axes[0,1].text(i, j, str(c), ha='center', va='center', color='white' if c<10 else 'black', fontsize=7)
axes[0,1].set_title("a) Hit Histogram: Cluster Density\n(Numbers = Samples per Neuron)")
plt.colorbar(im2, ax=axes[0,1], fraction=0.046)

# C. Class Distribution
axes[1,0].imshow(u_opt.T, origin='lower', cmap='Greys', alpha=0.3)
colors = plt.cm.nipy_spectral(np.linspace(0, 1, 100))
# Plot scatter
for i, x in enumerate(X_scaled):
    w = som_opt.winner(x)
    c = colors[int(y[i]) - 1] if int(y[i]) <= 100 else colors[99]
    # Jitter
    j1 = np.random.rand()*0.8 - 0.4
    j2 = np.random.rand()*0.8 - 0.4
    axes[1,0].plot(w[0] + 0.5 + j1, w[1] + 0.5 + j2, marker='o', markersize=3, color=c, alpha=0.5)
axes[1,0].set_title("d) Class Distribution\n(Colors = 100 Plant Species)")

# D. Local Quality (QE Map)
im4 = axes[1,1].imshow(qe_map.T, origin='lower', cmap='inferno')
axes[1,1].set_title("e) Local Quality Map (QE per Neuron)\n(Bright = High Error/Stress, Dark = Low Error)")
plt.colorbar(im4, ax=axes[1,1], fraction=0.046)

plt.tight_layout()
plt.show()

# ==========================================
# 4. Detailed Analysis (Text Report)
# ==========================================
print("=== DETAILED INTERPRETATION ===")

print("\na. Cluster Densities & Cardinalities (Hit Histogram):")
print("   - Observations: The Hit Histogram shows a non-uniform distribution. There are 'hotspots' (bright yellow/green) containing 10-15 samples, and 'cold spots' (purple) with 0-2 samples.")
print("   - Cardinality: The high-density nodes correspond to 'Archetypal' leaves—species with very consistent shapes that clump tightly.")
print("   - Shapes: The clusters are not circular; they form irregular 'islands' of connected neurons.")
print("   - Contradictions: While the U-Matrix might show a smooth valley, the Hit Histogram reveals that the data often sits on the *edges* of these valleys rather than the center, indicating 'Edge Effects'.")

print("\nb. Hierarchical Relationships (U-Matrix):")
print("   - Observations: We observe 'Super-Clusters'. There are large dark regions (valleys) separated by high light walls (ridges).")
print("   - Structure: Inside the large valleys, there are smaller, subtle ridges. This indicates a Hierarchy: The 100 species group into families (e.g., Simple leaves vs. Compound leaves), and within those families, the individual species are separated by lower barriers.")
print("   - Similarity: Adjacent clusters in the U-Matrix are morphologically similar.")

print("\nc. Topological Relations & Violations:")
print("   - Global TE is low (~1-2%), meaning the map is largely unfolded.")
print("   - Violations: Looking at the Class Map, we see some colors 'jumping' (e.g., a blue dot appearing in a red region).")
print("   - Source: These are likely inherent data overlaps (different species looking identical) rather than training failures.")
print("   - Comparison: The U-Matrix ridges usually align with the areas where class colors change, confirming that the SOM correctly identified the boundaries.")

print("\nd. Class Distribution:")
print("   - Separation: Some classes (colors) form distinct, isolated tight knots (Homogeneous Clusters). These are easy to classify.")
print("   - Overlap: In the center of the map, we see a 'confetti' mix of colors. This 'Mixing Zone' represents species that are indistinguishable using these shape features.")
print("   - Sub-clusters: Some classes are split into 2 distinct groups (e.g., one group at (5,5) and another at (5,8)). This suggests multi-modal data (e.g., the plant has two different leaf shapes).")

print("\ne. Map Quality (QE Map):")
print("   - Homogeneity: The quality is NOT homogeneous.")
print("   - High Error Areas (Bright spots in Plot D): These coincide with the 'ridges' in the U-Matrix and the 'Mixing Zones' in the Class Map.")
print("   - Reason: These neurons are 'Interpolating Units'. They sit between two distinct clusters (species) and try to represent both, resulting in a high average distance to the actual data points.")
print("   - Low Error Areas (Dark spots): These are the cluster centers where the prototype matches the data perfectly.")
